In [ ]:
###############
### install ###
###############
'''

pip install pandas

'''

In [ ]:
###########################################
### delete tweet lines without location ###
###########################################
import pandas as pd

low_memory=False
df = pd.read_csv('data_twitter_api.csv',engine='python', error_bad_lines=False)   #reading of the two dataset, data with tweet and data of the city
dfcity = pd.read_csv('data_city_update.csv')

try:                                                                               #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    dfcity.pop('Unnamed: 0')                                                       #Because pd.read_csv creates a column 'Unnamed: 0'
except KeyError as err:
        pass

to_fast=len(df)                                                                    #if the data contains too many tweets we can access it by reducing the data by modifying len(df) by the number of data we want
df_fast = df[:to_fast]

i=0                                                                                 #The index of the first element 

while i<len(df_fast):
    try :
        if ',' in df_fast.user_location[i]:                             
            text =df_fast.user_location[i]
            head, sep, tail = text.partition(',')
            df_fast.user_location[i]=head                                          #Removing the comma and after from the user_location column

    except TypeError as err:                                                       #A way to handle the error. If the program encounters a TypeError, it will pass and continue the program
        pass

    i+=1                                                                           #add 1 to e to keep the index updated


e=0                                                                                #The index of the first element 
twi=[]                                                                             #creation of an empty list

for i in df_fast['user_location']:                                                 #loop that will look at each location of user_location in the tweet data set
    
    for a in dfcity['city']:                                                       #another loop that will look at each location of city in the dataset of the world city
        if i == a:                                                                 #check whether the two elements are the same
            twi.append(e)                                                          #if yes, we add e (index element) of the line in data set of tweet
    e+=1                                                                           #add 1 to e to keep the index updated
    
twi = list(set(twi))                                                               #many cities in the world have the same name so we delete duplicate elements
twi.sort()                                                                         #It sorts the list of index of the tweet with the correct location

nb_tweet_in_data = len(twi)-1                                                      #Taking the last index of the list of the index of the tweet with the correct location
df_s = df_fast[:twi[nb_tweet_in_data]]

to_drop = [ele for ele in range(len(df_s)) if ele not in twi]                      #we take the inverse of the elements to know which are the elements without corect localization in order to remove them from the data
to_drop.sort()

for i in (to_drop):                                                                #loop that will drop the line without location in the dataset
    df_s=df_s.drop(i)
    

'''
with open("listTwi.txt", "w") as f:       #Writing the list of the index of the tweet with the correct location in a txt file
    for s in twi:
        f.write(str(s) +",")
'''

df_s.to_csv('data_5K.csv')                                                          #Saving the dataframe df_s to a csv file called data_5K.csv


In [ ]:
##########################################################################
### Adding the latitude and longitude for each location for each tweet ###
##########################################################################

import pandas as pd

df = pd.read_csv('data_5K.csv')                #Reading the csv file
dfcity = pd.read_csv('data_city_update.csv')   #Reading the csv file

try:                                           #Trying to remove the column 'Unnamed: 0' from the dataframe dfcity. If it fails, it will pass. Because pd.read_csv creates a column 'Unnamed: 0'
    df.pop('Unnamed: 0')
except KeyError as err:
        pass

e=0                                            ##The index of the first element
latitude=[]                                    #creation of an empty list which will be used to indicate the latitude assigned to the city of a tweet
longitude=[]                                   #creation of an empty list which will be used to indicate the longitude assigned to the city of a tweet


for location in df['user_location']:           #This is a loop that is going through the user_location column of the dataframe df
    while location != dfcity.city[e]:          #comparing it to the city column of the dataframe dfcity
        e+=1                                   #add 1 to e to keep the index updated
    latitude.append(dfcity.lat[e])             #if the while loop stops it means that the right city is found, the latitude and longitude of the city is appended to the latitude and longitude list
    longitude.append(dfcity.lng[e])
    


    e=0                                        #reset the index to 0 as the loop starts again
    

df.insert(6,"U_latitude",latitude,True)        #Inserting the latitude and longitude list into the dataframe df at the 6th and 7th column
df.insert(7,"U_longitude",longitude,True)


df.to_csv('data_tweet_with_location.csv')      #Saving the dataframe df into a csv file called tweet_with_location.csv